In [3]:
# !pip install gigachat
# !pip install gigachain

In [4]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials='ZjFkZGJmOTgtZmJhMS00NDlhLTkzODUtMjQyNzk3OGIwYzgwOjk0NjQ4NGZlLTlkZGYtNDNiOC05NmQ2LTFkMDYzMGY4ODljZQ==', verify_ssl_certs=False)

messages = [
    SystemMessage(
        content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
    )
]

while(True):
    # Ввод пользователя
    user_input = input("User: ")
    messages.append(HumanMessage(content=user_input))
    res = chat(messages)
    messages.append(res)
    # Ответ модели
    print("Bot: ", res.content)

/home/marsel/miniforge3/envs/gigachat/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Bot:  Здравствуйте! Я готов помочь вам решить ваши проблемы.
Bot:  Два нуба, два друга, 
Два программиста, 
Два человека, 
Что хотят создать бота. 
Они будут работать вместе, 
Чтобы сделать его умным, 
И они будут учиться вместе, 
Чтобы сделать его полезным. 
Они будут делиться знаниями, 
Они будут помогать друг другу, 
Они будут поддерживать друг друга, 
И они будут создавать что-то великое.
